In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams

import re

In [64]:
df = pd.read_csv("C:/Users/manth/Project/Text Classification for Hotel Review/train (2).csv")
df.head()

ID                                             Review  Rating
0   0  exceptional service nice all-around daughter s...       5
1   1  beautiful relaxing jw marriott desert ridge re...       5
2   2  great location great location 5 mins subway ta...       5
3   3  pleased nice safe hotel, flower market hotel v...       3
4   4  excellent hotel service great hotel excellent ...       4

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14343 entries, 0 to 14342
Data columns (total 3 columns):
ID        14343 non-null int64
Review    14343 non-null object
Rating    14343 non-null int64
dtypes: int64(2), object(1)
memory usage: 336.3+ KB


In [66]:
df.isnull().sum()

ID        0
Review    0
Rating    0
dtype: int64

In [67]:
print(df.Rating.value_counts())

5    6436
4    4172
3    1510
2    1248
1     977
Name: Rating, dtype: int64


In [68]:
df.Rating.describe()

count    14343.000000
mean         3.965070
std          1.230439
min          1.000000
25%          3.000000
50%          4.000000
75%          5.000000
max          5.000000
Name: Rating, dtype: float64

In [69]:
df.groupby('Rating').describe()

ID                                                             \
         count         mean          std   min      25%     50%       75%   
Rating                                                                      
1        977.0  7138.492323  4130.144108  32.0  3577.00  6917.0  10751.00   
2       1248.0  7089.788462  4150.087921  13.0  3466.50  7050.5  10632.00   
3       1510.0  7101.452318  4114.071624   3.0  3497.25  7286.0  10661.25   
4       4172.0  7251.357622  4141.163144   4.0  3686.50  7296.5  10911.50   
5       6436.0  7155.909416  4146.643874   0.0  3566.75  7136.5  10708.25   

                 
            max  
Rating           
1       14342.0  
2       14330.0  
3       14341.0  
4       14340.0  
5       14339.0

In [71]:
# text preprocessing

def preprocessing (document):
    
    document = str(document).lower() # convert to lower cases
    document = re.sub(r'\d+', '', document) # remove numbers
    document = document.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    document = document.strip() # remove white space
    
    tokenizer = RegexpTokenizer(r'\w+') # tokenize
    tokens = tokenizer.tokenize(document)
    
    # stop words
    stop_words = stopwords.words('english')  
    filtered_words = [w for w in tokens if not w in stop_words] # remove stop words
        
    # generate bigrams
    bigrams = ngrams(tokens, 2)
    # convert to list of bigram lists
    bigrams_clean = [list(generator) for generator in bigrams]
    # combine stopword list
    stop_words = stopwords.words('english')  
   
    # remove bigram if BOTH words are in stopword list
    bigrams_clean = [bigram for bigram in bigrams_clean if not all(word in stop_words for word in bigram)]
    # stem bigrams
    wordnet_lemmatizer = WordNetLemmatizer()
    bigrams_clean = [(wordnet_lemmatizer.lemmatize(bigram[0]),wordnet_lemmatizer.lemmatize(bigram[1])) for bigram in bigrams_clean]
        
        
        # stemmer
        #stemmer = EnglishStemmer()
        #filtered_words_stem = [stemmer.stem(w) for w in filtered_words]
        
    # Lemmatization 
    wordnet_lemmatizer = WordNetLemmatizer()
    filtered_words_lem = [wordnet_lemmatizer.lemmatize(w) for w in filtered_words]
                
    return (filtered_words_lem, bigrams_clean)

In [76]:
results = [preprocessing(reviews) for reviews in df['Review']]
cleaned_df = pd.DataFrame(results, columns=['unigram_clean', 'bigrams_clean'])

In [77]:
df1 = pd.concat([df, cleaned_df], axis=1)
df1.head()

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   
3   3  pleased nice safe hotel, flower market hotel v...       3   
4   4  excellent hotel service great hotel excellent ...       4   

                                       unigram_clean  \
0  [exceptional, service, nice, allaround, daught...   
1  [beautiful, relaxing, jw, marriott, desert, ri...   
2  [great, location, great, location, min, subway...   
3  [pleased, nice, safe, hotel, flower, market, h...   
4  [excellent, hotel, service, great, hotel, exce...   

                                       bigrams_clean  
0  [(exceptional, service), (service, nice), (nic...  
1  [(beautiful, relaxing), (relaxing, jw), (jw, m...  
2  [(great, location), (location, great), (great,...  
3  [(pleased, nice), (nice, safe), (safe, hotel),...  
4  [(excellent, hotel), (hotel, service), (servic...